In [47]:
import numpy as np
import pandas as pd
import glob
import os
import datetime


In [48]:
# get data from all user directories
dataset_dir = '../cache/real_world_dataset/'



In [71]:
df_activities_data = None
df_context_data = None
for user_dir in glob.glob(f"{dataset_dir}/p*"):
    user_id = user_dir.split("/")[-1]
    labels_files = glob.glob(f'{user_dir}/exported/*')
    print(user_id, labels_files)
    for filepath in labels_files:
        filedir = filepath.split("/")[-1].split(".")[0]
        session_id = user_id + '_' + filedir.split("_session_")[-1]
        session_start = filedir.split("_session_")[0]
        session_start = pd.to_datetime(session_start, format='%Y_%m_%d-%I_%M_%S_%p')
        # print(session_id, session_start)
        df_session = pd.read_csv(filepath,sep='\t',names=['label_type','nan','start_time','end_time','duration','label'])
        df_session['start_time'] = pd.to_datetime(df_session['start_time'],format='%H:%M:%S.%f')
        df_session['start_time'] = df_session['start_time'].apply(lambda x: session_start + datetime.timedelta(hours=x.hour,minutes=x.minute,seconds=x.second,microseconds=x.microsecond))
        df_session['end_time'] = pd.to_datetime(df_session['end_time'],format='%H:%M:%S.%f')
        # df_session['duration'] = pd.to_datetime(df_session['duration'],format='%H:%M:%S.%f')
        df_session['end_time'] = df_session['end_time'].apply(lambda x: session_start + datetime.timedelta(hours=x.hour,minutes=x.minute,seconds=x.second,microseconds=x.microsecond))
        df_session['session_id'] = f"{session_id}"
        df_session['user_id'] = f"{user_id}"
        df_session['session_start'] = session_start
        df_session['start_timestamp'] = pd.to_numeric(df_session['start_time'].values) //10**9
        df_session['end_timestamp'] = pd.to_numeric(df_session['end_time'].values) //10**9

        df_activities = df_session[df_session.label_type=='activity'][['user_id','session_id','start_timestamp','end_timestamp','label']]
        df_contexts = df_session[df_session.label_type=='context'][['user_id','session_id','start_timestamp','end_timestamp','label']]
        if df_activities_data is None:
            df_activities_data = df_activities.copy(deep=True)
        else:
            df_activities_data = pd.concat([df_activities_data,df_activities],ignore_index=True)

        if df_context_data is None:
            df_context_data = df_contexts.copy(deep=True)
        else:
            df_context_data = pd.concat([df_context_data,df_contexts],ignore_index=True)
df_activities_data = df_activities_data[~df_activities_data.label.isnull()]
df_context_data = df_context_data[~df_context_data.label.isnull()]

p4 ['../cache/real_world_dataset/p4/exported/2022_09_06-04_19_24_PM_session_3.txt', '../cache/real_world_dataset/p4/exported/2022_09_06-04_28_23_PM_session_4.txt', '../cache/real_world_dataset/p4/exported/2022_09_06-04_37_09_PM_session_1.txt', '../cache/real_world_dataset/p4/exported/2022_09_06-04_08_11_PM_session_2.txt']
p3 ['../cache/real_world_dataset/p3/exported/2022_09_07-03_44_11_PM_session_1.txt', '../cache/real_world_dataset/p3/exported/2022_09_07-04_11_54_PM_session_4.txt', '../cache/real_world_dataset/p3/exported/2022_09_07-03_52_27_PM_session_2.txt', '../cache/real_world_dataset/p3/exported/2022_09_07-04_02_43_PM_session_3.txt']
p2 ['../cache/real_world_dataset/p2/exported/2022_09_06-12_33_46_PM_session_2.txt', '../cache/real_world_dataset/p2/exported/2022_09_06-12_44_54_PM_session_3.txt', '../cache/real_world_dataset/p2/exported/2022_09_06-01_03_36_PM_session_1.txt', '../cache/real_world_dataset/p2/exported/2022_09_06-12_54_38_PM_session_4.txt']
p5 ['../cache/real_world_dat

In [72]:
df_context_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 462 entries, 0 to 462
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          462 non-null    object
 1   session_id       462 non-null    object
 2   start_timestamp  462 non-null    int64 
 3   end_timestamp    462 non-null    int64 
 4   label            462 non-null    object
dtypes: int64(2), object(3)
memory usage: 21.7+ KB


In [73]:
df_activities_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1149 entries, 0 to 1148
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          1149 non-null   object
 1   session_id       1149 non-null   object
 2   start_timestamp  1149 non-null   int64 
 3   end_timestamp    1149 non-null   int64 
 4   label            1149 non-null   object
dtypes: int64(2), object(3)
memory usage: 53.9+ KB


In [74]:
df_activities_data.head(20)


,user_id,session_id,start_timestamp,end_timestamp,label
0,p4,p4_3,1662481164,1662481192,noevent
1,p4,p4_3,1662481192,1662481200,jumping
2,p4,p4_3,1662481200,1662481207,running
3,p4,p4_3,1662481207,1662481233,noevent
4,p4,p4_3,1662481233,1662481240,mouse_click
5,p4,p4_3,1662481240,1662481246,typing
6,p4,p4_3,1662481246,1662481253,writing
7,p4,p4_3,1662481253,1662481266,noevent
8,p4,p4_3,1662481266,1662481272,running
9,p4,p4_3,1662481272,1662481279,jumping


In [75]:
df_activities_data.to_csv("../cache/real_world_dataset/df_activities.csv",index=False)
df_context_data.to_csv("../cache/real_world_dataset/df_contexts.csv",index=False)

In [76]:
print(','.join(df_activities_data.label.unique().tolist()))
#correct names
df_activities_data['label'] = df_activities_data['label'].apply(lambda x: x.strip().replace("wriitng","writing").replace("mouse_click","mouseclick").replace("door_knock",'doorknock').replace("door_open","dooropen"))
df_activities_data['label'] = df_activities_data['label'].apply(lambda x: x.strip().replace("phone_ring","phonering").replace("coffee_machine","coffeemachine").replace("no-event",'noevent').replace("noevemt","noevent").replace("noeevnt","noevent").replace(" ",""))
df_activities_data['label'] = df_activities_data['label'].apply(lambda x: x.strip().replace("houseWork","vacuum").replace("mouseclick","mouse").replace("sweping","sweeping"))
activity_list = ','.join(df_activities_data.label.unique().tolist())
activity_list = np.unique(activity_list.split(",")).tolist()
activity_list

noevent,jumping,running,mouse_click,typing,writing,wriitng,tv,tv, talking,tv, typing,tv, eating,tv, mouse_click,vacuum, talking,door_knock,door_open,phone_ring,talking,sweeping,vacuum,eating,coffee_machine,talking, sweeping,talking, coffee_machine,talking,sweeping,noevent , noevent,talking,coffee_machine,noeevnt,sweping,vacuum ,tv ,vacuum, talking  ,tv, typing ,noevent   ,tv, mouse,tv,eating,typing ,tv,eating ,tv, mouseclick,no-event,noevemt


['coffeemachine',
 'doorknock',
 'dooropen',
 'eating',
 'jumping',
 'mouse',
 'noevent',
 'phonering',
 'running',
 'sweeping',
 'talking',
 'tv',
 'typing',
 'vacuum',
 'writing']

In [77]:
len(activity_list)

15

In [78]:
# get final dataset for real world activities
df_activity_ts = None
for session_id in df_activities_data['session_id'].unique():
    df_session_data = df_activities_data[df_activities_data.session_id==session_id]
    min_ts,max_ts = df_session_data.start_timestamp.min(), df_session_data.end_timestamp.max()
    df_session_ts = pd.DataFrame(np.arange(min_ts, max_ts), columns=['timestamp'])
    df_session_ts['activity'] = ''
    df_session_ts = df_session_ts.set_index('timestamp')
    for row_idx, row in df_session_data.iterrows():
        df_session_ts.loc[row['start_timestamp']:row['end_timestamp']-1, 'activity'] = df_session_ts.loc[
                                                                        row['start_timestamp']:row['end_timestamp']-1,
                                                                        'activity'].apply(
            lambda x: row['label'] if (x == '') else (x + ',' + row['label']))
    df_session_ts = df_session_ts.reset_index()
    df_session_ts['session_id'] = session_id
    df_session_ts  = df_session_ts[['session_id','timestamp','activity']]
    print(f"session {session_id} ts shape",df_session_ts.shape)
    if df_activity_ts is None:
        df_activity_ts = df_session_ts.copy(deep=True)
    else:
        df_activity_ts = pd.concat([df_activity_ts,df_session_ts],ignore_index=True)



session p4_3 ts shape (338, 3)
session p4_4 ts shape (309, 3)
session p4_1 ts shape (230, 3)
session p4_2 ts shape (428, 3)
session p3_1 ts shape (293, 3)
session p3_4 ts shape (226, 3)
session p3_2 ts shape (390, 3)
session p3_3 ts shape (348, 3)
session p2_2 ts shape (415, 3)
session p2_3 ts shape (363, 3)
session p2_1 ts shape (252, 3)
session p2_4 ts shape (340, 3)
session p5_1 ts shape (294, 3)
session p5_3 ts shape (358, 3)
session p5_4 ts shape (308, 3)
session p5_2 ts shape (269, 3)
session p10_4 ts shape (283, 3)
session p10_1 ts shape (325, 3)
session p10_2 ts shape (399, 3)
session p10_3 ts shape (432, 3)
session p7_2 ts shape (300, 3)
session p7_1 ts shape (254, 3)
session p7_4 ts shape (326, 3)
session p7_3 ts shape (348, 3)
session p6_3 ts shape (412, 3)
session p6_4 ts shape (274, 3)
session p6_1 ts shape (292, 3)
session p6_2 ts shape (368, 3)
session p1_4 ts shape (295, 3)
session p1_1 ts shape (233, 3)
session p1_3 ts shape (352, 3)
session p1_2 ts shape (391, 3)


In [79]:
df_activity_ts.info()
df_activity_ts.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10445 entries, 0 to 10444
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   session_id  10445 non-null  object
 1   timestamp   10445 non-null  int64 
 2   activity    10445 non-null  object
dtypes: int64(1), object(2)
memory usage: 244.9+ KB


,session_id,timestamp,activity
0,p4_3,1662481164,noevent
1,p4_3,1662481165,noevent
2,p4_3,1662481166,noevent
3,p4_3,1662481167,noevent
4,p4_3,1662481168,noevent


In [80]:
df_activity_ts = df_activity_ts.sort_values(by=['session_id','timestamp'])
df_activity_ts = df_activity_ts[~(df_activity_ts.activity=='')]
df_activity_ts.to_csv('../cache/realworld_activity_log_gt.csv',index=False)

In [81]:
# getting mites activity ts
df_activity_mites_ts = pd.read_csv("../cache/real_world_dataset/gt_mites_labels.csv", names=['event1','event2','event3','label','session_id','timestamp','user_id'])
df_activity_mites_ts.event1 = df_activity_mites_ts.event1.fillna('')
df_activity_mites_ts.event2 = df_activity_mites_ts.event2.fillna('')
df_activity_mites_ts.event3 = df_activity_mites_ts.event3.fillna('')
df_activity_mites_ts['activity'] = df_activity_mites_ts['event1'] + ',' + df_activity_mites_ts['event2'] + ',' + df_activity_mites_ts['event3']
df_activity_mites_ts.head()

,event1,event2,event3,label,session_id,timestamp,user_id,activity
0,noevent,,,noevent,p4_3,1.662481e+12,p4,"noevent,,"
1,noevent,,,noevent,p4_3,1.662481e+12,p4,"noevent,,"
2,noevent,,,noevent,p4_3,1.662481e+12,p4,"noevent,,"
3,noevent,,,noevent,p4_3,1.662481e+12,p4,"noevent,,"
4,noevent,,,noevent,p4_3,1.662481e+12,p4,"noevent,,"


In [82]:
# df_activity_ts.activity.unique().tolist()
df_activity_ts[df_activity_ts.activity=='']

,session_id,timestamp,activity


In [83]:
for activity in activity_list:
    df_activity_ts[activity] = df_activity_ts['activity'].apply(lambda x: activity in x)
    df_activity_mites_ts[activity] = df_activity_mites_ts['activity'].apply(lambda x: activity in x)
df_activity_ts = df_activity_ts.drop('activity',axis=1)
df_activity_ts.info()
df_activity_mites_ts.info()
df_activity_ts.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10444 entries, 5444 to 7479
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   session_id     10444 non-null  object
 1   timestamp      10444 non-null  int64 
 2   coffeemachine  10444 non-null  bool  
 3   doorknock      10444 non-null  bool  
 4   dooropen       10444 non-null  bool  
 5   eating         10444 non-null  bool  
 6   jumping        10444 non-null  bool  
 7   mouse          10444 non-null  bool  
 8   noevent        10444 non-null  bool  
 9   phonering      10444 non-null  bool  
 10  running        10444 non-null  bool  
 11  sweeping       10444 non-null  bool  
 12  talking        10444 non-null  bool  
 13  tv             10444 non-null  bool  
 14  typing         10444 non-null  bool  
 15  vacuum         10444 non-null  bool  
 16  writing        10444 non-null  bool  
dtypes: bool(15), int64(1), object(1)
memory usage: 397.8+ KB
<class 'pa

,session_id,timestamp,coffeemachine,doorknock,dooropen,eating,jumping,mouse,noevent,phonering,running,sweeping,talking,tv,typing,vacuum,writing
5444,p10_1,1662667543,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
5445,p10_1,1662667544,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
5446,p10_1,1662667545,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
5447,p10_1,1662667546,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
5448,p10_1,1662667547,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False


In [84]:
df_activity_ts

,session_id,timestamp,coffeemachine,doorknock,dooropen,eating,jumping,mouse,noevent,phonering,running,sweeping,talking,tv,typing,vacuum,writing
5444,p10_1,1662667543,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
5445,p10_1,1662667544,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
5446,p10_1,1662667545,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
5447,p10_1,1662667546,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
5448,p10_1,1662667547,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7475,p7_4,1662497413,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
7476,p7_4,1662497414,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
7477,p7_4,1662497415,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
7478,p7_4,1662497416,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False


In [85]:
df_activity_mites_ts

,event1,event2,event3,label,session_id,timestamp,user_id,activity,coffeemachine,doorknock,...,mouse,noevent,phonering,running,sweeping,talking,tv,typing,vacuum,writing
0,noevent,,,noevent,p4_3,1.662481e+12,p4,"noevent,,",False,False,...,False,True,False,False,False,False,False,False,False,False
1,noevent,,,noevent,p4_3,1.662481e+12,p4,"noevent,,",False,False,...,False,True,False,False,False,False,False,False,False,False
2,noevent,,,noevent,p4_3,1.662481e+12,p4,"noevent,,",False,False,...,False,True,False,False,False,False,False,False,False,False
3,noevent,,,noevent,p4_3,1.662481e+12,p4,"noevent,,",False,False,...,False,True,False,False,False,False,False,False,False,False
4,noevent,,,noevent,p4_3,1.662481e+12,p4,"noevent,,",False,False,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348741,noevent,,,noevent,p1_2,1.662486e+12,p1,"noevent,,",False,False,...,False,True,False,False,False,False,False,False,False,False
348742,noevent,,,noevent,p1_2,1.662486e+12,p1,"noevent,,",False,False,...,False,True,False,False,False,False,False,False,False,False
348743,noevent,,,noevent,p1_2,1.662486e+12,p1,"noevent,,",False,False,...,False,True,False,False,False,False,False,False,False,False
348744,noevent,,,noevent,p1_2,1.662486e+12,p1,"noevent,,",False,False,...,False,True,False,False,False,False,False,False,False,False


In [86]:
activity_list

['coffeemachine',
 'doorknock',
 'dooropen',
 'eating',
 'jumping',
 'mouse',
 'noevent',
 'phonering',
 'running',
 'sweeping',
 'talking',
 'tv',
 'typing',
 'vacuum',
 'writing']

In [43]:
df_activity_ts.to_csv("../datasets/realworld_dataset.csv",index=False)
df_activity_mites_ts.to_csv("../datasets/realworld_mites_dataset.csv",index=False)

In [87]:
df_activity_mites_ts['timestamp'] = df_activity_mites_ts['timestamp']//10**3
df_activity_mites_ts['timestamp'] = df_activity_mites_ts['timestamp'].astype(int)
df_activity_mites_ts['timestamp'].min(),df_activity_mites_ts['timestamp'].max()

(1662467626, 1662669655)

In [88]:
#create leave one out dataset for activities
for id in df_activities_data.user_id.unique():
    df_id_activity_ts = df_activity_ts.copy(deep=True)
    df_id_activity_ts['isTrain'] = True
    df_id_activity_ts.loc[df_id_activity_ts.session_id.str.contains(id),'isTrain'] = False
    # df_id_activity_ts.info()

    df_id_mites_activity_ts = df_activity_mites_ts.copy(deep=True)
    df_id_mites_activity_ts['isTrain'] = True
    df_id_mites_activity_ts.loc[df_id_mites_activity_ts.session_id.str.contains(id),'isTrain'] = False
    print(df_id_mites_activity_ts.timestamp.max())

    df_id_activity_ts.to_csv(f"../datasets/realworld_lopo/{id}_dataset.csv",index=False)
    df_id_mites_activity_ts.to_csv(f"../datasets/realworld_lopo_mites/{id}_dataset.csv",index=False)


1662669655
1662669655
1662669655
1662669655
1662669655
1662669655
1662669655
1662669655


In [127]:
df_id_activity_ts.head()

,session_id,timestamp,coffeemachine,doorknock,dooropen,eating,jumping,mouse,noevent,phonering,running,sweeping,talking,tv,typing,vacuum,writing,isTrain
5444,p10_1,1662667543,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
5445,p10_1,1662667544,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
5446,p10_1,1662667545,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
5447,p10_1,1662667546,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False
5448,p10_1,1662667547,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False


In [67]:
df_id_mites_activity_ts.head()

,event1,event2,event3,label,session_id,timestamp,user_id,activity,coffeemachine,doorknock,...,noevent,phonering,running,sweeping,talking,tv,typing,vacuum,writing,isTrain
0,noevent,,,noevent,p4_3,1.662481e+09,p4,"noevent,,",False,False,...,True,False,False,False,False,False,False,False,False,True
1,noevent,,,noevent,p4_3,1.662481e+09,p4,"noevent,,",False,False,...,True,False,False,False,False,False,False,False,False,True
2,noevent,,,noevent,p4_3,1.662481e+09,p4,"noevent,,",False,False,...,True,False,False,False,False,False,False,False,False,True
3,noevent,,,noevent,p4_3,1.662481e+09,p4,"noevent,,",False,False,...,True,False,False,False,False,False,False,False,False,True
4,noevent,,,noevent,p4_3,1.662481e+09,p4,"noevent,,",False,False,...,True,False,False,False,False,False,False,False,False,True


In [120]:
# real world contexts
df_context_data.head()

,user_id,session_id,start_timestamp,end_timestamp,label
0,p4,p4_3,1662481192,1662481208,exercising
1,p4,p4_3,1662481233,1662481253,officeWork
2,p4,p4_3,1662481266,1662481279,exercising
3,p4,p4_3,1662481304,1662481318,exercising
4,p4,p4_3,1662481327,1662481340,exercising


In [121]:
df_context_data.label.unique()

array(['exercising', 'officeWork', 'relax', 'relax, officeWork',
       'relax, havingMeal', 'houseWork, officeWork', 'person_entered',
       'onAPhoneCall', 'houseWork', 'havingMeal, officeWork',
       'mealPreparation', 'onAPhoneCall, houseWork',
       'onAPhoneCall, mealPreparation', 'havingMeal,officeWork',
       'onAPhoneCall, Mealpreparation', 'housework',
       'onAPhoneCall,houseWork', 'onAPhoneCall,mealPreparation',
       'person_entered ', 'onAPhoneCall ', 'houseWork ', 'exercising ',
       'officeWork  ', 'havingMeal, OfficeWork',
       'havingMeal, OfficeWork ', 'officeWork ', ' officeWork',
       'onAPhoneCall, houseWork ', 'OnPhoneACall', 'OnPhoneCall',
       'OnAPhoneCall', 'HouseWork', 'havingMeal',
       'havingMeal, officeWork ', 'person_entered  ',
       'relax, officeWork ', 'onAPhoneCall, officeWork',
       'havingMEal, officeWork', 'onAPhoneCall h'], dtype=object)

In [89]:
print(','.join(df_context_data.label.unique().tolist()))
#correct names
df_context_data['label'] = df_context_data['label'].apply(lambda x: x.strip().replace("exercising","Exercising").replace("relax","Relaxing").replace("officeWork",'OfficeWork').replace("havingMeal","HavingMeal"))
df_context_data['label'] = df_context_data['label'].apply(lambda x: x.strip().replace(" ","").replace("OnAPhoneCall","PhoneCall").replace("houseWork",'HouseWork').replace("mealPreparation","MealPreparation"))
df_context_data['label'] = df_context_data['label'].apply(lambda x: x.strip().replace("person_entered","ComingIn").replace("OnPhoneACall","PhoneCall").replace("PhoneCallh",'PhoneCall').replace("Mealpreparation","MealPreparation"))
df_context_data['label'] = df_context_data['label'].apply(lambda x: x.strip().replace("havingMEal","HavingMeal").replace("housework","HouseWork").replace("onAPhoneCall","PhoneCall").replace("OnPhoneCall","PhoneCall"))
# df_context_data['label'] = df_context_data['label'].apply(lambda x: x.strip().replace("phone_ring","phonering").replace("coffee_machine","coffeemachine").replace("no-event",'noevent').replace("noevemt","noevent").replace(" ",""))
context_list = ','.join(df_context_data.label.unique().tolist())
context_list = np.unique(context_list.split(",")).tolist()
context_list

exercising,officeWork,relax,relax, officeWork,relax, havingMeal,houseWork, officeWork,person_entered,onAPhoneCall,houseWork,havingMeal, officeWork,mealPreparation,onAPhoneCall, houseWork,onAPhoneCall, mealPreparation,havingMeal,officeWork,onAPhoneCall, Mealpreparation,housework,onAPhoneCall,houseWork,onAPhoneCall,mealPreparation,person_entered ,onAPhoneCall ,houseWork ,exercising ,officeWork  ,havingMeal, OfficeWork,havingMeal, OfficeWork ,officeWork , officeWork,onAPhoneCall, houseWork ,OnPhoneACall,OnPhoneCall,OnAPhoneCall,HouseWork,havingMeal,havingMeal, officeWork ,person_entered  ,relax, officeWork ,onAPhoneCall, officeWork,havingMEal, officeWork,onAPhoneCall h


['ComingIn',
 'Exercising',
 'HavingMeal',
 'HouseWork',
 'MealPreparation',
 'OfficeWork',
 'PhoneCall',
 'Relaxing']

In [90]:
df_context_data.head()
df_context_data = df_context_data[['session_id','start_timestamp','end_timestamp','label']]
df_context_data.columns = ['id','start_time','end_time','context']
df_context_data.head()

,id,start_time,end_time,context
0,p4_3,1662481192,1662481208,Exercising
1,p4_3,1662481233,1662481253,OfficeWork
2,p4_3,1662481266,1662481279,Exercising
3,p4_3,1662481304,1662481318,Exercising
4,p4_3,1662481327,1662481340,Exercising


In [124]:
df_context_data.to_csv("../GT_marking/gt_realworld.csv",index=False)

In [91]:
df_context_data.context.drop_duplicates()

0                     Exercising
1                     OfficeWork
10                      Relaxing
11           Relaxing,OfficeWork
15           Relaxing,HavingMeal
19          HouseWork,OfficeWork
35                      ComingIn
38                     PhoneCall
41                     HouseWork
44         HavingMeal,OfficeWork
45               MealPreparation
51           PhoneCall,HouseWork
52     PhoneCall,MealPreparation
270                   HavingMeal
393         PhoneCall,OfficeWork
Name: context, dtype: object

In [92]:
df_activity_mites_ts.head()

,event1,event2,event3,label,session_id,timestamp,user_id,activity,coffeemachine,doorknock,...,mouse,noevent,phonering,running,sweeping,talking,tv,typing,vacuum,writing
0,noevent,,,noevent,p4_3,1662481164,p4,"noevent,,",False,False,...,False,True,False,False,False,False,False,False,False,False
1,noevent,,,noevent,p4_3,1662481164,p4,"noevent,,",False,False,...,False,True,False,False,False,False,False,False,False,False
2,noevent,,,noevent,p4_3,1662481164,p4,"noevent,,",False,False,...,False,True,False,False,False,False,False,False,False,False
3,noevent,,,noevent,p4_3,1662481164,p4,"noevent,,",False,False,...,False,True,False,False,False,False,False,False,False,False
4,noevent,,,noevent,p4_3,1662481164,p4,"noevent,,",False,False,...,False,True,False,False,False,False,False,False,False,False


In [102]:
df_mites_activity_ts = df_activity_mites_ts[['session_id','timestamp'] + activity_list]
df_mites_activity_ts = df_mites_activity_ts[~df_mites_activity_ts.noevent]
df_mites_activity_ts.head()

,session_id,timestamp,coffeemachine,doorknock,dooropen,eating,jumping,mouse,noevent,phonering,running,sweeping,talking,tv,typing,vacuum,writing
182,p4_3,1662481169,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
183,p4_3,1662481169,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
184,p4_3,1662481169,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
185,p4_3,1662481169,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
186,p4_3,1662481169,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False


In [103]:
df_mites_activity_ts = df_mites_activity_ts.groupby(['session_id','timestamp'],as_index=False).sum()
df_mites_activity_ts

,session_id,timestamp,coffeemachine,doorknock,dooropen,eating,jumping,mouse,noevent,phonering,running,sweeping,talking,tv,typing,vacuum,writing
0,p10_1,1662667548,0,0,0,0,0,0,0,0,0,0,21,0,0,0,0
1,p10_1,1662667549,0,0,0,0,0,0,0,0,0,0,13,0,0,0,0
2,p10_1,1662667556,0,0,0,0,0,0,0,0,0,0,23,0,0,0,0
3,p10_1,1662667557,0,3,0,0,0,0,0,0,0,0,23,0,0,0,0
4,p10_1,1662667558,0,28,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6314,p7_4,1662497414,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0
6315,p7_4,1662497415,0,0,0,0,0,0,0,0,0,0,0,34,0,0,0
6316,p7_4,1662497416,0,0,0,0,0,0,0,0,0,0,0,33,0,0,0
6317,p7_4,1662497417,0,0,0,0,0,0,0,10,0,0,0,26,0,0,0


In [104]:
df_mites_activity_ts['activity'] = '-'
for activity in activity_list:
    df_mites_activity_ts['activity'] = df_mites_activity_ts.apply(lambda row: (row['activity']+','+activity) if row[activity]>0. else row['activity'],axis=1)

df_mites_activity_ts

,session_id,timestamp,coffeemachine,doorknock,dooropen,eating,jumping,mouse,noevent,phonering,running,sweeping,talking,tv,typing,vacuum,writing,activity
0,p10_1,1662667548,0,0,0,0,0,0,0,0,0,0,21,0,0,0,0,"-,talking"
1,p10_1,1662667549,0,0,0,0,0,0,0,0,0,0,13,0,0,0,0,"-,talking"
2,p10_1,1662667556,0,0,0,0,0,0,0,0,0,0,23,0,0,0,0,"-,talking"
3,p10_1,1662667557,0,3,0,0,0,0,0,0,0,0,23,0,0,0,0,"-,doorknock,talking"
4,p10_1,1662667558,0,28,0,0,0,0,0,0,0,0,0,0,0,0,0,"-,doorknock"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6314,p7_4,1662497414,0,0,0,0,0,0,0,0,0,0,0,8,0,0,0,"-,tv"
6315,p7_4,1662497415,0,0,0,0,0,0,0,0,0,0,0,34,0,0,0,"-,tv"
6316,p7_4,1662497416,0,0,0,0,0,0,0,0,0,0,0,33,0,0,0,"-,tv"
6317,p7_4,1662497417,0,0,0,0,0,0,0,10,0,0,0,26,0,0,0,"-,phonering,tv"


In [105]:
df_mites_activity_ts = df_mites_activity_ts[['session_id','timestamp','activity']]
df_mites_activity_ts['activity'] = df_mites_activity_ts['activity'].str.replace("-,","").replace("_","")
df_mites_activity_ts['activity'].unique().tolist()

<ipython-input-105-8024e89f4b2a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mites_activity_ts['activity'] = df_mites_activity_ts['activity'].str.replace("-,","").replace("_","")


['talking',
 'doorknock,talking',
 'doorknock',
 'running,talking',
 'doorknock,dooropen',
 'mouse',
 'dooropen,sweeping',
 'coffeemachine,doorknock',
 'coffeemachine,doorknock,jumping,running,typing',
 'coffeemachine,doorknock,running',
 'coffeemachine,eating,jumping,running,typing',
 'coffeemachine,dooropen,eating,running',
 'coffeemachine,dooropen,eating,running,typing',
 'coffeemachine,running,typing',
 'coffeemachine,eating,jumping,running',
 'coffeemachine,jumping,running,typing',
 'coffeemachine,dooropen,running,talking,typing',
 'coffeemachine,running,talking,typing',
 'coffeemachine,jumping',
 'coffeemachine,jumping,typing',
 'coffeemachine,jumping,running',
 'sweeping',
 'talking,typing,writing',
 'writing',
 'phonering',
 'typing',
 'phonering,talking',
 'talking,tv',
 'tv',
 'doorknock,phonering',
 'dooropen',
 'typing,writing',
 'eating',
 'doorknock,eating',
 'doorknock,eating,talking',
 'coffeemachine',
 'coffeemachine,sweeping',
 'talking,typing',
 'coffeemachine,jumpin

In [106]:
df_mites_activity_ts.to_csv("../cache/df_mites_activity_ts.csv",index=False)